In [1]:
import csv
from nltk import word_tokenize, ngrams,WordNetLemmatizer,PorterStemmer,sent_tokenize # had to download punkt from cmd line - import nltk nltk.download()
from nltk.corpus import wordnet as wordn #show if the word is an adjective, noun, verb or adverb
from collections import Counter
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import re
import os
import datetime
from textblob import TextBlob
import statistics
english_sw = stopwords.words('english') # same as comment above for stopwords corpus


# stopwords have some useful stuff in when taken in the context of multi-grams
english_sw_redact = english_sw

for z in ['during','against','below','before', 'after', 'above','yourself', 'yourselves',
          'ourselves','over', 'under', 'again', 'further','until', 'while']:
    english_sw_redact.remove(z)
    
    
extra_stop = ['us','really','would','make','get','say','also',
              'made','many','take','over','under','going','let',
              'put','may','see','give','back','every','given',
              'done','go','before','two','next','across','says',
              'come','could','per','since','however','after','much',
              'again','able','whether','taken','cent','even','extra','mr','set','still','ago','thing','asked','got',
              'last','hour','one','two','three','four','five','six','seven','eight','nine','ten',
              'monday','tuesday','wednesday','thursday','friday','saturday','sunday','day','today','yesterday',
              'january','february','march','april','june','july','august','september','october','november','december'
]    
english_sw.extend(extra_stop)


lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
sia = SentimentIntensityAnalyzer()
loca = "\\Analysis\\" # where to save the processed NGrams files.
_pmqmaster = "ALLCONTRIBSMASTER -CAT.txt" # Raw Transcript log file from Hansard API

donedata = []
with open(loca + 'AllDataPBI.txt','r') as latestfl:
    latestfl.readline()
    lrdr = csv.reader(latestfl,delimiter='\t')
    for lrow in lrdr:
        donedata.append(lrow)

donemonths = [d[2] for d in donedata]
donemonths = list(set(donemonths))
donemonths = [datetime.datetime.strptime(d,"%Y-%m-%d") for d in donemonths]
lastmonth = max(donemonths)
lastmonthtext = datetime.datetime.strftime(lastmonth,"%Y-%m-%d")

donedata = [d for d in donedata if d[2] != lastmonthtext] # lop off last month of done data
donemonths = [d for d in donemonths if d != lastmonth]




In [2]:
lastmonth

datetime.datetime(2023, 7, 1, 0, 0)

In [2]:
conslist = []
rejects = []
regs = ['\[[\w\W\s]{1,25}\]','\([\w\W\s]{1,25}\)','^rose\u2014','^rose\-','^rose$']
exceptions = ['(a) are in prison and (b)','(a) disposal, (b)']
with open(_pmqmaster,'r') as fl:
    line = fl.readline()
    line = fl.readline()
    cnt = 1
    while line:
        
        rej = []
        
        lal = line.strip().split('\t')
        rowlist = [cnt]
        rowlist.extend(lal[3:5])
        yrmonth = datetime.datetime.strptime(lal[4],'%d/%m/%Y')
        rowlist.append(datetime.datetime(yrmonth.year,yrmonth.month,1))#monthyear
        rowlist.extend(lal[5:])
        #conslist.append(rowlist)
        spc = lal[3]
        for rx in regs:
            res = re.findall(rx,spc)
            rej.extend(res)
        if len(rej) > 0:
            #rejects.append([cnt,rej])
            rejunique = [x for x in list(set(rej)) if x not in exceptions]
            for rejterm in rejunique:
                spc = spc.replace(rejterm,'')
        if re.search('\w',spc):
            rowlist[1] = spc
            conslist.append(rowlist)
        else:
            rejects.append(rowlist)
        cnt += 1
        line = fl.readline()


        
print("Processed raw data into list var")      

def tokensprep(tst):
    #tst is a list of words
    #connect spoken lang orphans (haven't), hon. as in honourable, and replace curly format double-quotations
    orphans = ["'s","'t", "'ve", "'ll", "'re","'d"]
    for i,v in enumerate(tst):
        for o in orphans:
            if o in v.lower():
        
                tst[i-1] = tst[i-1] + v
                tst.remove(tst[i])
                break
                
    for i,v in enumerate(tst):
        if tst[i] == '.' and tst[i-1].lower() == 'hon':
            tst[i-1] = tst[i-1] + v
            tst.remove(tst[i])
    
    
    for t in tst:
        t = t.replace('\u201d','"').replace('\u201c','"')
    
    return tst
        
def ngrams_useful(x,y,bigram_switch=False):
    #return True if the ngram (as list var x) has no punctuation as one of the words, and has less than ...
    #... var y number of REDACTED stopwords. Should stop 'I am' appearing as bigram
    check = True
    ct = 0
    # flag invalid if punctuation
    for wrd in x:
        if not re.search('\w',wrd):
            check = False
            break
    #build stopword score
    for wrd in x:
        if wrd.lower() in english_sw_redact:
            ct +=1
            
    if ct >= y:
        check=False
        
    if bigram_switch: #if running on bigrams, both components can't be in the extended stopwords, and no numbers 
        ctb = 0
        for wrd in x:
            if wrd.lower() in english_sw:
                ctb +=1
            elif re.search('[0-9]',wrd.lower()):
                ctb +=2
        if ctb >=2:
            check=False
    return check        
        
def smart_lemma(x):
    try:
        wordtype = wordn.synsets(x)[0].pos() # get the first pos tag of the word. 'a' and 's' are adjectives,'v'=verb,'r'=adverb,'n'=noun
        if wordtype == 'n':
            out = lemmatizer.lemmatize(x) #noun is default for lemmatizer. THis should standardise plurals
        elif wordtype == 'v':
            out = lemmatizer.lemmatize(x, pos='v') #standardize verbs
        else:
            #leave alone
            out = x
    except:
        out = x
    return out        
        
def ngramlist(x,y,swmax,bgswitch=False):
    #return list of distinct ngrams with counts, where x is a list of speeches, y is n grams, and swmax is the limit...
    #... to the number of REDACTED stopwords that can appear in the gram for it to be counted and returned.
    #bgswitch tells the function that it is running on bigrams - so the ngrams_useful call should factor this in and check
    #...both words in bigrams against the extendend stopwords
    gramslist = []
    for spch in x:
        
        if y > 1:
            toks = tokensprep(list(word_tokenize(spch)))
            rawgrams = [list(b) for b in list(ngrams(toks,y))]
            cleangrams = [re.sub('\.$',''," ".join(c).lower()) for c in rawgrams if ngrams_useful(c,swmax,bgswitch)]
            gramslist.extend(cleangrams)
        else:
            toks = tokensprep(list(word_tokenize(spch.replace('…',' '))))
            cleangrams = [re.sub('\.$','',tko.lower()) for tko in toks if not re.search('[^a-z]', tko.lower()) and tko.lower() not in english_sw]
            if len(cleangrams) > 0:
                cleangramslemma = [smart_lemma(cl) for cl in cleangrams]
                gramslist.extend(cleangramslemma)
        
    ngram_cts = Counter(gramslist)
    return [[i[0],i[1]] for i in ngram_cts.items()]

#WordCount
def wordcount(x):
    gramslist = []
    for spch in x:
        toks = tokensprep(list(word_tokenize(spch.replace('…',' '))))
        cleangrams = [re.sub('\.$','',tko.lower()) for tko in toks if re.search('\w',tko)]
        gramslist.extend(cleangrams)
    return (len(gramslist),len(x))


rejects = None

months = list(set([x[3] for x in conslist if x[3] not in donemonths]))
monthscts = list(set([x[3] for x in conslist]))
roles = list(set([x[4] for x in conslist]))
parties = list(set([x[5] for x in conslist]))
people = list(set([x[-1] for x in conslist if x[4] != 'BB' and len(x) ==7]))

print("Starting ngrams")

configsettings = [[1,1,1,'SingleIII.txt'],[2,1,1,'BGIII.txt'],[3,1,1,'TrigramsII.txt'],
                  [4,2,1,'4_grams.txt'],[5,3,1,'5_grams.txt'],[6,3,1,'6_grams.txt'],[7,3,1,'7_grams.txt']]


for c_st in configsettings:
    masterlist = []
    _n = c_st[0]
    _swmax = c_st[1]
    min_occ = c_st[2]
    if _n==2: #propagate into ngrams checking
        masterbgswitch = True
    else:
        masterbgswitch = False
    for mo in months:
        
        analytime = [x for x in conslist if x[3] == mo]
        for ro in roles:
            analyroletime = [x for x in analytime if x[4] == ro]
            for p in parties:
                analyroletimepty = [x for x in analyroletime if x[5] == p]
                inds = []
                gen_bbs = []
                for aaa in analyroletimepty:
                    
                    if ro != 'BB':
                        inds.append(aaa)
                    elif len(aaa)==7:
                        if aaa[6] in people:
                            
                            inds.append(aaa)
                        else:
                            gen_bbs.append(aaa[1])
                    else:
                        gen_bbs.append(aaa[1])
                        
                #now have gen_bbs (can be done at once) and inds(needs to be looped thru person by person)
                    
                #operate function on gen_bbs
                if len(gen_bbs)> 0:
                    outgr_data_bbs = ngramlist(gen_bbs,_n,_swmax,masterbgswitch)
                    if len(outgr_data_bbs) > 0:
                        with_cats_bbs = [[x[0],x[1],mo,ro,p,''] for x in outgr_data_bbs]
                        masterlist.extend(with_cats_bbs)
                    
                #for each person...
                for ppl in people:
                    #testlist = [x[2:] for x in inds if x[6] == ppl]
                    #if len(testlist) > 0:
                    #    for t in testlist:
                    #    
                    #        masterlist.append(t)
                    personspecific = [x[1] for x in inds if x[6] == ppl]
                    #operate on person specific
                    if len(personspecific) > 0: #there is data for this person in this orle at this month
                        #print(personspecific)
                        outgr_data = ngramlist(personspecific,_n,_swmax,masterbgswitch)
                        
                        if len(outgr_data) > 0:
                            
                            with_cats = [[x[0],x[1],mo,ro,p,ppl] for x in outgr_data]
                            masterlist.extend(with_cats)
                    
    
    print('All {0} ngrams: '.format(_n), len(masterlist))                
    #remove any that are single across whole file                
    masterlistcts = {}
    for m in masterlist:
        trm = m[0]
        ct = m[1]
        if trm in masterlistcts:
            masterlistcts[trm] += ct
        else:
            masterlistcts[trm] = ct
    print('Dict done')        
    disallowlist = [[x,1] for x in masterlistcts if masterlistcts[x] < min_occ]
    print('Disallow done',len(disallowlist))
    masterdf = pd.DataFrame(masterlist, columns=['Value', 'Count','YearMonth', 'Role','Party','Person'])
    disallowdf = pd.DataFrame(disallowlist, columns=['ValueDis','Disallow'])
    mergedf = pd.merge(masterdf, disallowdf, left_on='Value', right_on='ValueDis',how='left',validate='m:1')
    neatdf = mergedf[mergedf['Disallow'] != 1].drop(['ValueDis','Disallow'],axis=1)
    loca = "C:\\Users\\nedst\\OneDrive\\Documents\\DataplusVisualPresentationII - LBAG\\UCOVI\\BLOG WORK\\Covid Post III - Parliament\\Debate Scrapes\\Analysis\\"
    neatdf.to_csv(loca + c_st[3],index=False,sep='\t')


print("Starting Tenure Stats and WOrd COunts")    
    
masterlist = []

for mo in monthscts:
    #print(mo)
    analytime = [x for x in conslist if x[3] == mo]
    for ro in roles:
        analyroletime = [x for x in analytime if x[4] == ro]
        for p in parties:
            analyroletimepty = [x for x in analyroletime if x[5] == p]
            inds = []
            gen_bbs = []
            for aaa in analyroletimepty:
                
                if ro != 'BB':
                    inds.append(aaa)
                elif len(aaa)==7:
                    if aaa[6] in people:
                        
                        inds.append(aaa)
                    else:
                        gen_bbs.append(aaa[1])
                else:
                    gen_bbs.append(aaa[1])
                    
            #now have gen_bbs (can be done at once) and inds(needs to be looped thru person by person)
                
            #operate function on gen_bbs
            if len(gen_bbs)> 0:
                wc,fre = wordcount(gen_bbs)
                
                with_cats_bbs = [wc,fre,None,mo,ro,p,'']
                masterlist.append(with_cats_bbs)
                
            #for each person...
            for ppl in people:
                #testlist = [x[2:] for x in inds if x[6] == ppl]
                #if len(testlist) > 0:
                #    for t in testlist:
                #    
                #        masterlist.append(t)
                personspecific = [x[1] for x in inds if x[6] == ppl]
                persondates = len(list(set([x[2] for x in inds if x[6] == ppl])))
                #operate on person specific
                if len(personspecific) > 0: #there is data for this person in this orle at this month
                    wcp,frep = wordcount(personspecific)

                    with_cats = [wcp,frep,persondates,mo,ro,p,ppl]
                    masterlist.append(with_cats)    

masterdf = pd.DataFrame(masterlist, columns=['Value', 'Count','DebatesSpokenIn','YearMonth', 'Role','Party','Person'])
masterdf.to_csv(loca + 'SpeechesAndWordCounts.txt',index=False,sep='\t')

tenures = []
for mo in monthscts:
    dates = list(set([x[2] for x in conslist if x[3] == mo]))
    for d in dates:
        distroles = list(set([x[4] for x in conslist if x[2] == d]))
        
        if 'Deputy Speaker' in distroles:
            dsflag = 1
        else:
            dsflag = 0
        if 'Dep PM (replacing)' in distroles:
            dpflag = 1
        else:
            dpflag = 0 
        
        if 'Dep LOE (replacing)' in distroles:
            dleflag = 1
        else:
            dleflag = 0             
        tenures.append([mo,d,dsflag,dpflag,dleflag])
        
maxtenuresdate = max([datetime.datetime.strptime(x[1],'%d/%m/%Y') for x in tenures])


pms = ['Tony Blair','Gordon Brown','David Cameron','Theresa May','Boris Johnson','Elizabeth Truss','Rishi Sunak']
loes = ['John Major','William Hague','Iain Duncan Smith','Michael Howard','David Cameron','Ed Miliband',
        'Jeremy Corbyn','Kier Starmer']

speakers_min_max = [['Betty Boothroyd', datetime.datetime.strptime('21/05/1997','%d/%m/%Y'),datetime.datetime.strptime('26/07/2000','%d/%m/%Y')],
                    ['Michael Martin',  datetime.datetime.strptime('25/10/2000','%d/%m/%Y'),datetime.datetime.strptime('17/06/2009','%d/%m/%Y')],
                    ['John Bercow',     datetime.datetime.strptime('24/06/2009','%d/%m/%Y'),datetime.datetime.strptime('30/10/2019','%d/%m/%Y')],
                    ['Lindsay Hoyle',   datetime.datetime.strptime('08/01/2020','%d/%m/%Y'),maxtenuresdate]]
pms_min_max = []
loes_min_max = []
conslistnames = [x for x in conslist if len(x) == 7]
for pm in pms:
    dates = [datetime.datetime.strptime(x[2],'%d/%m/%Y') for x in conslistnames if x[4] == 'PM' and x[6] == pm]
    pms_min_max.append([pm,min(dates),
                        max(dates)])  
for lo in loes:
    dates = [datetime.datetime.strptime(x[2],'%d/%m/%Y') for x in conslistnames if x[4] == 'LOE' and x[6] == lo]
    loes_min_max.append([lo,min(dates),
                        max(dates)])
    
for te in tenures:
    sitpm = None
    sitloe = None
    sitsp = None
    dt = datetime.datetime.strptime(te[1],'%d/%m/%Y')
    for p in pms_min_max:
        if dt >= p[1] and dt <= p[2]:
            sitpm = p[0]
            break
    for p in loes_min_max:
        if dt >= p[1] and dt <= p[2]:
            sitloe = p[0]
            break
    for p in speakers_min_max:
        if dt >= p[1] and dt <= p[2]:
            sitsp = p[0]
            break 
                
    # current leaders of opposition
    if sitloe is None and te[0].year in [2021,2022,2023]:
        sitloe = 'Kier Starmer'
    if sitpm is None and te[0].year in [2022,2023]:
        sitpm = 'Rishi Sunak'
    
    #
    
    
    te.append(sitsp)
    te.append(sitpm)
    te.append(sitloe)
    
masterdf = pd.DataFrame(tenures, columns=['Month', 'Date',
                                          'DepSpeaker','DepPM','DepLOE', 
                                          'SittingSpeaker','SittingPM','SittingLOE',])

masterdf.to_csv(loca + 'DebateInfo.txt',index=False,sep='\t')


print("Filemerge")


filestom = [
 ['4_grams.txt','4-Grams'],
 ['5_grams.txt','5-Grams'],
 ['6_grams.txt','6-Grams'],
 ['7_grams.txt','7-Grams'],
 ['BGIII.txt','2.Bigrams'],
 ['SingleIII.txt','1.SingleWord'],
 ['TrigramsII.txt','3.Trigrams']
]

mergeurl = loca + 'AllDataPBI.txt'


with open(mergeurl,"w",newline='') as mu:
    wrt = csv.writer(mu,delimiter='\t')
    wrt.writerow(['Value','Count','YearMonth','Role','Party','Person','TextType'])
    
    for v in filestom:
        textype = v[1]
        with open(loca + v[0],"r") as raw:
            
            
            raw.readline()
            rdr = csv.reader(raw,delimiter='\t')
            for rw in rdr:
                rw.append(textype)
                wrt.writerow(rw)
                
print("Appending done data")
with open(mergeurl,"a",newline='') as mut:
    wrtt = csv.writer(mut,delimiter='\t')

    
    for rwt in donedata:

        wrtt.writerow(rwt)            
                
print("Sentiment")



Processed raw data into list var
Starting ngrams
All 1 ngrams:  7231
Dict done
Disallow done 0
All 2 ngrams:  3688
Dict done
Disallow done 0
All 3 ngrams:  1313
Dict done
Disallow done 0
All 4 ngrams:  4253
Dict done
Disallow done 0
All 5 ngrams:  7490
Dict done
Disallow done 0
All 6 ngrams:  3785
Dict done
Disallow done 0
All 7 ngrams:  1675
Dict done
Disallow done 0
Starting Tenure Stats and WOrd COunts
Filemerge
Appending done data
Sentiment


In [3]:

def full_senti_calc(x):
    
    sentences = sent_tokenize(x)
    if len(sentences) == 0:
        return {'TextBlob':None,'NltkVader':None}
    tbscorelist = []
    vadscorelist = []
    for s in sentences:
        wc = len(word_tokenize(s))
        weight = 1.0 + ((float(wc) ** float(1/3)) * 0.1) #add 10% of cube root of word length as weight
        tbscore = TextBlob(s).sentiment.polarity
        vader_score = sia.polarity_scores(s)['compound']
        tbscorelist.append(round(tbscore*weight,4))
        vadscorelist.append(round(vader_score*weight,4))
        
        #print(s,'\n','TextBlob: ',round(tbscore,4),'NLTK Vader: ',round(vader_score,4),'  ', weight)
    return {'TextBlob':round(statistics.mean(tbscorelist),4),
            'NltkVader':round(statistics.mean(vadscorelist),4)}

def issue_tag(x,xdate,xtaglist):
    tags = []
    spacerep = re.sub('[\.,:;\?!"]',' ',x).lower()
    for trm in xtaglist:
        if trm[0] in spacerep and xdate.year >= trm[2]:
            tags.append(trm[1])
    return sorted(list(set(tags)))

tagsfile = loca + 'TaggingPhrases.txt'
tagslist = []
tagslistclean = []
with open(tagsfile,'r') as tgfl:
    line = tgfl.readline()
    line = tgfl.readline()
    while line:
        rw = line.strip().split('\t')
        tagslist.append(rw)
        line = tgfl.readline()


for t in tagslist:
    tspace = ' ' + t[0] + ' '
    reg = tspace.replace('* ','')
    tag = t[1]
    if len(t) == 2:
        year = 1996
    else:
        year = int(t[2])
        
    tagslistclean.append([reg,tag,year])
    
    
conslist_light = []
conslist_tags = []
conslist_tagged = []
rollct = 0
for c in conslist:
    rollct += 1
    if rollct % 3000 == 0:
        print(rollct)
    _id = c[0]
    _text = c[1]
    _day = c[2]
    _month = c[3]
    _role,_party = c[4],c[5]
    if len(c) == 7:
        _person = c[6]
    else:
        _person = None
    issues = issue_tag(_text,_month,tagslistclean)
    _sentiments = full_senti_calc(_text)
    
    conslist_light.append([_id,_day,_month,_role,_party,_person

                           ,_sentiments['TextBlob'],_sentiments['NltkVader']])

    
    
    
    if len(issues) > 0:
        conslist_tags.extend([[_id,x] for x in issues])
        conslist_tagged.append([_id,_month,_role,_party,_person
                               ,_text])  
        
with open(loca + 'ContributionsSentiment.txt','w',newline='') as wr:
    wrt = csv.writer(wr,delimiter='\t')
    wrt.writerow(['ID','DebateDate','DebateMonth','Role','Party','Person'
                  #,'Text'])
                  ,'TextBlob','NltkVader'])
    for x in conslist_light:
        wrt.writerow(x)
        
with open(loca + 'Tags.txt','w',newline='') as wr:
    wrt = csv.writer(wr,delimiter='\t')
    wrt.writerow(['ConstributionID','Tag'])
    for x in conslist_tags:
        wrt.writerow(x)    
        
        
with open(loca + 'TaggedContributionsSentiment.txt','w',newline='') as wr:
    wrt = csv.writer(wr,delimiter='\t')
    wrt.writerow(['ID','DebateMonth','Role','Party','Person'
                  ,'Text'])

    for x in conslist_tagged:
        wrt.writerow(x)       
        


3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000


In [4]:
conslist_light = None
conslist_tags = None
conslist_tagged = None
conslist = None
masterlist = None
sentence_word_counts = None
donedata = None